# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data = Path(r"C:\Users\laris\Documents\python-api-challenge\python-api-challenge\WeatherPy\output_data\cities.csv")
city_data_df = pd.read_csv(city_data)

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,7.40,95,93,14.13,TF,1710002220
1,1,thompson,55.7435,-97.8558,-12.91,72,20,1.03,CA,1710002747
2,2,college,64.8569,-147.8028,-14.82,78,100,0.00,US,1710002747
3,3,hawaiian paradise park,19.5933,-154.9731,19.46,97,100,3.60,US,1710002233
4,4,hasaki,35.7333,140.8333,1.22,49,1,8.13,JP,1710002748


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
# Configure the map plot

map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    color = "City",
    alpha = 0.5,
    size = "Humidity",
    scale = 1,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
    )

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
#A max temperature lower than 27 degrees but higher than 21
#Wind speed less than 4.5 m/s
#Zero cloudiness

weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 27)
                                       & (city_data_df["Max Temp"] > 21) 
                                       & (city_data_df["Cloudiness"] == 0)
                                       & (city_data_df["Wind Speed"] < 4.5)
                                      ]

# Drop any rows with null values
weather_df.dropna()

# Display sample data
weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,hermanus,-34.4187,19.2345,23.22,82,0,2.88,ZA,1710002226
47,47,kailua-kona,19.6406,-155.9956,23.11,85,0,2.06,US,1710002081
65,65,lady frere,-31.7031,27.2329,22.08,65,0,4.35,ZA,1710002755
131,131,kruisfontein,-34.0033,24.7314,22.16,79,0,2.62,ZA,1710002764
161,161,callao,-12.0667,-77.1500,26.16,61,0,3.60,PE,1710002308
204,204,aliwal north,-30.6937,26.7114,24.99,14,0,2.02,ZA,1710002775
273,273,amlagora,22.8333,87.3333,22.86,22,0,2.63,IN,1710002785
315,315,pisco,-13.7000,-76.2167,26.68,69,0,3.60,PE,1710002791
403,403,holualoa,19.6228,-155.9522,21.04,85,0,2.06,US,1710002232
407,407,murud,18.3258,72.9650,25.18,55,0,3.37,IN,1710002804


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
11,hermanus,ZA,-34.4187,19.2345,82,
47,kailua-kona,US,19.6406,-155.9956,85,
65,lady frere,ZA,-31.7031,27.2329,65,
131,kruisfontein,ZA,-34.0033,24.7314,79,
161,callao,PE,-12.0667,-77.1500,61,
204,aliwal north,ZA,-30.6937,26.7114,14,
273,amlagora,IN,22.8333,87.3333,22,
315,pisco,PE,-13.7000,-76.2167,69,
403,holualoa,US,19.6228,-155.9522,85,
407,murud,IN,18.3258,72.9650,55,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
          "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    longitude = row['Lng']
    latitude = row['Lat']
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)

    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hermanus - nearest hotel: Aloe guest house
kailua-kona - nearest hotel: Kona Seaside Hotel
lady frere - nearest hotel: No hotel found
kruisfontein - nearest hotel: No hotel found
callao - nearest hotel: Casa Ronald
aliwal north - nearest hotel: No hotel found
amlagora - nearest hotel: No hotel found
pisco - nearest hotel: La Portada
holualoa - nearest hotel: Kona Hotel
murud - nearest hotel: No hotel found
villa regina - nearest hotel: Nuevo Cheo
shwebo - nearest hotel: Pyi Shwe Theingha


,City,Country,Lat,Lng,Humidity,Hotel Name
11,hermanus,ZA,-34.4187,19.2345,82,Aloe guest house
47,kailua-kona,US,19.6406,-155.9956,85,Kona Seaside Hotel
65,lady frere,ZA,-31.7031,27.2329,65,No hotel found
131,kruisfontein,ZA,-34.0033,24.7314,79,No hotel found
161,callao,PE,-12.0667,-77.1500,61,Casa Ronald
204,aliwal north,ZA,-30.6937,26.7114,14,No hotel found
273,amlagora,IN,22.8333,87.3333,22,No hotel found
315,pisco,PE,-13.7000,-76.2167,69,La Portada
403,holualoa,US,19.6228,-155.9522,85,Kona Hotel
407,murud,IN,18.3258,72.9650,55,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
     geo = True,
     color = "City",
     alpha = 0.5,
     size = "Humidity",
     tiles = "OSM",
     hover_cols = ["Hotel Name", "Country"],
     frame_width = 700,
     frame_height = 500
     )

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)